# Getting started with Ensemblify

Ensemblify is a library used for generating and analyzing protein conformational ensembles of proteins. In this notebook you can find some examples of its usage.

In [ ]:
# Uncomment for running in Google Colab
#!pip install ensemblify

In [ ]:
# meter imports aqui

In [ ]:
# Import Ensemblify library
import ensemblify as ey

# Setting up your conformational sampling parameters file

# Generating a conformational ensemble

In [ ]:
# Uncomment for running in Google Colab
#! curl -OJL https://github.com/npfernandes/ensemblify/raw/main/examples/input_structures/Hst5.pdb

In [1]:
from ensemblify.reweighting.ensemble_utils import attempt_read_reweigthing_data

attempt_read_reweigthing_data('/home/tiagogomes/Desktop/projects/nuno_fernandes/NProtein_sarscov2/NProtein_365_TetramerClosed_Ensemble/testing_hst5_anal/reweighting',
                              'Hst5')

['Lagrange multipliers initialized from zero\n', 'Optimizing 1043 data and 10000 samples. Theta=1 \n', 'CHI2 before optimization:   1.9652 \n', 'Execution time: 25.36 seconds\n', 'Minimization using L-BFGS-B successful (iterations:142)\n', 'CHI2 after optimization:   0.9915 \n', 'Fraction of effective frames:   0.2434 \n']
['Lagrange multipliers initialized from zero\n', 'Optimizing 1043 data and 10000 samples. Theta=10 \n', 'CHI2 before optimization:   1.9652 \n', 'Execution time: 9.10 seconds\n', 'Minimization using L-BFGS-B successful (iterations:51)\n', 'CHI2 after optimization:   0.9960 \n', 'Fraction of effective frames:   0.4528 \n']
['Lagrange multipliers initialized from zero\n', 'Optimizing 1043 data and 10000 samples. Theta=20 \n', 'CHI2 before optimization:   1.9659 \n', 'Execution time: 7.21 seconds\n', 'Minimization using L-BFGS-B successful (iterations:45)\n', 'CHI2 after optimization:   0.9988 \n', 'Fraction of effective frames:   0.5205 \n']
['Lagrange multipliers init

('/home/tiagogomes/Desktop/projects/nuno_fernandes/NProtein_sarscov2/NProtein_365_TetramerClosed_Ensemble/testing_hst5_anal/reweighting/Hst5_exp_saxs.dat',
 '/home/tiagogomes/Desktop/projects/nuno_fernandes/NProtein_sarscov2/NProtein_365_TetramerClosed_Ensemble/testing_hst5_anal/reweighting/Hst5_calc_saxs.dat',
 array([    1,    10,    20,    20,    50,    75,   100,   200,   400,
          750,  1000,  5000, 10000]),
 array([[1.9652, 0.9915, 0.2434],
        [1.9652, 0.996 , 0.4528],
        [1.9659, 0.9988, 0.5205],
        [1.9659, 0.9988, 0.5205],
        [1.9652, 1.0099, 0.6147],
        [1.9652, 1.0178, 0.6572],
        [1.9652, 1.0254, 0.6882],
        [1.9652, 1.0528, 0.7605],
        [1.9652, 1.0972, 0.8249],
        [1.9652, 1.1626, 0.8775],
        [1.9652, 1.2045, 0.8999],
        [1.9652, 1.5637, 0.9828],
        [1.9652, 1.7148, 0.9941]]),
 array([[1.6874e-04, 7.0617e-05, 5.1591e-05, ..., 1.9897e-04, 1.1012e-04,
         9.4854e-05],
        [1.1384e-04, 6.2872e-05, 7.409